Creacion de las funciones antes de pasarlas a la API y creacion de las tablas de lectura para cada funcion con el fin de agiliar las consultas

In [1]:
import pandas as pd

In [2]:
#Lectura de ñps 3csv's
reviews = 'dfReviewsLimpio.csv'
items = 'dfItemsLimpio.csv'
steamGames= 'steamGamesLimpio.csv'
dfReviews = pd.read_csv(reviews)
dfItems = pd.read_csv(items)
dfSteamGames = pd.read_csv(steamGames)

In [5]:
dfReviews

,user_id,reviews.funny,reviews.posted,reviews.last_edited,reviews.item_id,reviews.helpful,reviews.recommend,sentimiento
0,76561197970982479,0,2011-11-05,NaN,1250,NaN,1,2
1,76561197970982479,0,2011-07-15,NaN,22200,NaN,1,2
2,76561197970982479,0,2011-04-21,NaN,43110,NaN,1,2
3,js41637,0,2014-06-24,NaN,251610,0.75,1,2
4,js41637,0,2013-09-08,NaN,227300,0.00,1,2
...,...,...,...,...,...,...,...,...
59300,76561198312638244,0,2012-07-10,NaN,70,NaN,1,2
59301,76561198312638244,0,2012-07-08,NaN,362890,NaN,1,2
59302,LydiaMorley,1,2012-07-03,NaN,273110,0.50,1,2
59303,LydiaMorley,0,2012-07-20,NaN,730,NaN,1,2


In [6]:
dfItems

,user_id,items_count,steam_id,item_id,item_name,playtime_2weeks,playtime_forever
0,76561197970982479,277,76561197970982479,10,Counter-Strike,0,6
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,30,Day of Defeat,0,7
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0,0
...,...,...,...,...,...,...,...
5153204,76561198329548331,7,76561198329548331,346330,BrainBread 2,0,0
5153205,76561198329548331,7,76561198329548331,373330,All Is Dust,0,0
5153206,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3,3
5153207,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4,4


In [7]:
dfSteamGames

,genres,title,release_date,early_access,id,user_id,years
0,Action,Lost Summoner Kitty,2018-01-04,0,761140,NaN,2018
1,Casual,Lost Summoner Kitty,2018-01-04,0,761140,NaN,2018
2,Indie,Lost Summoner Kitty,2018-01-04,0,761140,NaN,2018
3,Simulation,Lost Summoner Kitty,2018-01-04,0,761140,NaN,2018
4,Strategy,Lost Summoner Kitty,2018-01-04,0,761140,NaN,2018
...,...,...,...,...,...,...,...
70697,Indie,Russian Roads,2018-01-04,0,610660,NaN,2018
70698,Racing,Russian Roads,2018-01-04,0,610660,NaN,2018
70699,Simulation,Russian Roads,2018-01-04,0,610660,NaN,2018
70700,Casual,EXIT 2 - Directions,2017-09-02,0,658870,NaN,2017


In [8]:
# Conversiones generales extras
#Renombrar las columnas para poder hacer el Merge
dfSteamGames = dfSteamGames.rename(columns={'id': 'item_id'},)
# Cambiado de nombre
dfSteamGames = dfReviews.rename(columns={'reviews.item_id': 'item_id'},)
#Eliminar user_id
dfSteamGames = dfSteamGames.drop(columns=['user_id'])
# Extraer año
dfReviews['reviews.posted']= pd.to_datetime(dfReviews['reviews.posted'], format='%Y-%m-%d')
dfReviews['yearsReviews'] = dfReviews['reviews.posted'].dt.year
#COnvertir a MinusculaTodo
dfReviews = dfReviews.apply(lambda x: x.astype(str).str.lower() if x.name in ['item_name'] else x)


1.-def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [10]:
#Filtrado a columnas minima para optimizar las consultas 
# Sumar en funcion de ID
dfItemsGroupItemIdF1 = dfItems.groupby('item_id')['playtime_forever'].sum().reset_index()
# Hacer el merg de las tablas
itemsSteamGamesF1 = pd.merge(dfItemsGroupItemIdF1, dfSteamGames, on='item_id', how='inner')
# agrupamos por año y genero y sumamos la columna de playtimeForever
agrupacionF1 = itemsSteamGamesF1.groupby(['years', 'genres'])['playtime_forever'].sum().reset_index()


In [13]:
agrupacionF1

,years,genres,playtime_forever
0,1983,Action,3582
1,1983,Adventure,3582
2,1983,Casual,3582
3,1984,Action,384
4,1984,Adventure,384
...,...,...,...
338,2018,Adventure,21
339,2018,Early Access,6128
340,2018,Indie,2327
341,2018,RPG,3822


In [38]:
#Exportacion de tabla de consulta F1
agrupacionF1.to_csv('agrupacionF1.csv', index=False)

In [39]:
def funcion1(valorF1):
    if valorF1 in agrupacionF1['genres'].values :

        # Extrae el valor maximo de la columnas playtime_forever
        valorMaximoF1 = agrupacionF1.loc[agrupacionF1['genres'] == valorF1, 'playtime_forever'].max()
        añoBuscadoF1 = agrupacionF1.loc[agrupacionF1['playtime_forever'] == valorMaximoF1, 'years'].values[0]
        textoF1 = print("{"+'"Año de lanzamiento con más horas jugadas para Género',valorF1+'" :',str(añoBuscadoF1)+"}")
    else:
        textoF1 ="Ingrese un genero correcto"    
    return textoF1


In [40]:
#Prueba de funcion
funcion1('Indie')

{"Año de lanzamiento con más horas jugadas para Género Indie" : 2006}


2.- def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

In [14]:
#Filtrado a columnas minima para optimizar las consultas

# Join de la columnas items y dfSteam games por la clave item_id
itemsSteamGamesF2 = pd.merge(dfItems, dfSteamGames, on='item_id', how='inner')
# Join de la columnas itemsSteamGamesF2 y reviews por el Used_id
itemsSteamGamesF2YReview = pd.merge(itemsSteamGamesF2, dfReviews, on='user_id', how='inner')
# Concatenar Columnas con optimizacion 
agrupacionF2= itemsSteamGamesF2YReview.groupby(['user_id', 'yearsReviews','genres'])['playtime_forever'].sum().reset_index()


In [15]:
agrupacionF2

,user_id,yearsReviews,genres,playtime_forever
0,--000--,2014,Action,139465
1,--000--,2014,Adventure,11722
2,--000--,2014,Casual,16135
3,--000--,2014,Early Access,531
4,--000--,2014,Free to Play,20444
...,...,...,...,...
385869,zzoptimuszz,2015,Massively Multiplayer,60122
385870,zzoptimuszz,2015,RPG,60018
385871,zzoptimuszz,2015,Simulation,1760
385872,zzoptimuszz,2015,Sports,106


In [43]:
#Exportacion de tabla de consulta f2
agrupacionF2.to_csv('agrupacionF2.csv', index=False)

In [44]:
# Introduzco el genero
def funcion2(valorF2):
    if valorF2 in agrupacionF2['genres'].values :
        # Solo sacar los valores para ese genero asociado
        generoBuscadoF2 = agrupacionF2[agrupacionF2['genres'] == valorF2]

        # Suma del total de horas jugadas por usuario
        horasPorUsuario = generoBuscadoF2.groupby('user_id')['playtime_forever'].sum().reset_index()

        # Encontrar el usuario que tiene más horas jugadas para el género dado
        usuario_max_horas = horasPorUsuario.loc[horasPorUsuario['playtime_forever'].idxmax(), 'user_id']
        #Horas Maximas
        horas_maximas = horasPorUsuario['playtime_forever'].max()
        # Con el usuario con mas horas 
        añosUsuarioMasHoras = generoBuscadoF2[generoBuscadoF2['user_id'] == usuario_max_horas]
        # Encontrar la acumlacion de horas por año
        acumulacion_horas_por_anio = añosUsuarioMasHoras.groupby('yearsReviews')['playtime_forever'].sum().reset_index()
        # Renombras los indicies del filtro
        acumulacion_horas_por_anio.rename(columns={'yearsReviews': 'Año', 'playtime_forever': 'horas Jugadas'}, inplace=True)
        # Diccionario para impresion
        lista_diccionarios = acumulacion_horas_por_anio.to_dict(orient='records') 
        #Impresion del resultado de la funcion 
        textoF2 = print("{Usuario con más horas jugadas para Género "+valorF2,":",usuario_max_horas,",","HorasJugadas",":", lista_diccionarios)
    else:
        textoF2 = "Ingrese Genero Correcto"
    return textoF2

In [45]:
funcion2('Action')

{Usuario con más horas jugadas para Género Action : Sp3ctre , 13541200 
 [{'Año': 2012, 'horas Jugadas': 6770600}, {'Año': 2014, 'horas Jugadas': 1692650}, {'Año': 2015, 'horas Jugadas': 5077950}]


def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [17]:
#Filtrado a columnas minima para optimizar las consultas
# Año, reviewsrecomend
# De ese año, dame los 3 juegos que tengan mas puntos positivos neutrales

# Seleccionar solo los que fueron recomendados
reviewsTrue = dfReviews[dfReviews['reviews.recommend'] == 1]

# Agrupar por item_id y año y sumame el sentimiento asociados al ipo de juego y año dadpo
agruF3PorItem= reviewsTrue.groupby(['item_id', 'yearsReviews'])['sentimiento'].sum().reset_index()

# Dropear duplicados de items'ID
dfIItemsDuplicatesF3 = dfItems.drop_duplicates(subset='item_id')

# Merge entre las agrupacion y dfIItemsDuplicates
agruF3Inicial = pd.merge(agruF3PorItem, dfIItemsDuplicatesF3, on='item_id', how='inner')

#cColumnas releventes
agrupacionF3 = agruF3Inicial[['yearsReviews','sentimiento','item_name']]


In [18]:
agrupacionF3

,yearsReviews,sentimiento,item_name
0,2011,2,Counter-Strike
1,2012,15,Counter-Strike
2,2013,7,Counter-Strike
3,2014,33,Counter-Strike
4,2015,30,Counter-Strike
...,...,...,...
5556,2012,4,True or False
5557,2012,2,Super Switch
5558,2012,0,You Have 10 Seconds 2
5559,2012,2,Galactic Storm


In [48]:
#Exportacion de tabla de consulta f3
agrupacionF3.to_csv('agrupacionF3.csv', index=False)

In [49]:
def funcion3(valorF3):
    # Condicional si exite el numero de año
    if valorF3 in agrupacionF3['yearsReviews'].values :
    #Filtro de año
        # Igualar un filtro de año, solo al año a tomar
        añoFiltroF3 = agrupacionF3[agrupacionF3['yearsReviews'] == valorF3]
        # Ordenar los valores de mayor a menor
        top3F3 = añoFiltroF3.sort_values(by='sentimiento', ascending=False).head(3)
        # Reiniciar el indice
        top3F3 = top3F3.reset_index(drop=True)
        # Crear la columnas puero 
        top3F3['Puesto'] = 'Puesto ' + (top3F3.index + 1).astype(str)
        # Tomar solo dos columnas relavantes para el diccionario de salida
        top3F3Ordenado = top3F3[['Puesto','item_name']]
        # Crear lista de diccionarios con indices respectivos
        lista_diccionariosF3= top3F3Ordenado.set_index('Puesto')['item_name'].to_dict()
        # Texto de salida funcio
        textoF3 = lista_diccionariosF3
    #Caso contrario 
    else:
        textoF3 = "Ingrese fecha Correcta"
    return textoF3

In [50]:
#Prueba de funcion f3
funcion3(2014)

{'Puesto 1': 'Counter-Strike: Global Offensive',
 'Puesto 2': "Garry's Mod",
 'Puesto 3': 'Unturned'}

In [51]:
#Prueba de funcion f3
funcion3(2014)

{'Puesto 1': 'Counter-Strike: Global Offensive',
 'Puesto 2': "Garry's Mod",
 'Puesto 3': 'Unturned'}

In [21]:
#Filtrado a columnas minima para optimizar las consultas
# Año, reviewsrecomend
# De ese año, dame los 3 juegos que tengan MENOS puntos positivos neutrales

# Seleccionar solo los que no fueron recomendados
reviewsFalsosF4= dfReviews[dfReviews['reviews.recommend'] == 0]

# Agrupar por item_id y año y sumame el sentimiento asociados al ipo de juego y año dadpo
agruF4PorItem= reviewsFalsosF4.groupby(['item_id', 'yearsReviews'])['sentimiento'].sum().reset_index()

# Dropear duplicados de items'ID
dfIItemsDuplicatesF4 = dfItems.drop_duplicates(subset='item_id')

# Merge entre las agrupacion y dfIItemsDuplicates
agruF4Inicial = pd.merge(agruF4PorItem, dfIItemsDuplicatesF4, on='item_id', how='inner')

#cColumnas releventes
agrupacionF4 = agruF4Inicial[['yearsReviews','sentimiento','item_name']]

In [22]:
agrupacionF4

,yearsReviews,sentimiento,item_name
0,2015,2,Counter-Strike
1,2012,0,Team Fortress Classic
2,2013,2,Team Fortress Classic
3,2014,2,Team Fortress Classic
4,2015,4,Team Fortress Classic
...,...,...,...
2200,2012,2,Sky Valley
2201,2012,0,SCP 087. Re
2202,2012,2,ROD: Revolt Of Defense
2203,2012,1,The Pirate: Caribbean Hunt


In [54]:
#Exportacion de tabla de consulta f4
agrupacionF4.to_csv('agrupacionF4.csv', index=False)

In [55]:
def funcion4(valorF4):
    # Condicional si exite el numero de año
    if valorF4 in agrupacionF4['yearsReviews'].values :
    #Filtro de año
        # Igualar un filtro de año, solo al año a tomar
        añoFiltroF4 = agrupacionF4[agrupacionF4['yearsReviews'] == valorF4]
        # Ordenar los valores de menor a mayor
        top3F4 = añoFiltroF4.sort_values(by='sentimiento', ascending=True).head(3)
        # Reiniciar el indice
        top3F4 = top3F4.reset_index(drop=True)
        # Crear la columnas puesto 
        top3F4['Puesto'] = 'Puesto ' + (top3F4.index + 1).astype(str)
        # Tomar solo dos columnas relevantes para el diccionario de salida
        top3F4Ordenado = top3F4[['Puesto','item_name']]
        # Crear lista de diccionarios con indices respectivos
        lista_diccionariosF4= top3F4Ordenado.set_index('Puesto')['item_name'].to_dict()
        # Texto de salida funcio
        textoF4 = lista_diccionariosF4
    #Caso contrario 
    else:
        textoF4 = "Ingrese fecha Correcta"
    return textoF4

In [56]:
#PREUBA DE FUNCION
funcion4(2012)

{'Puesto 1': 'Team Fortress Classic',
 'Puesto 2': 'Super Chibi Knight',
 'Puesto 3': '100% Orange Juice'}

In [57]:
#PREUBA DE FUNCION
funcion4(2012)

{'Puesto 1': 'Team Fortress Classic',
 'Puesto 2': 'Super Chibi Knight',
 'Puesto 3': '100% Orange Juice'}

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

In [24]:
#Filtrado a columnas minima para optimizar las consultas
# Igual el dfReviews a un nuevo dataframe
dfconValF5 = dfReviews
#Colocar disgregar en columnas para negativo, positivo y neutral
dfconValF5['Negativo'] = dfconValF5['sentimiento'].apply(lambda x: 1 if x == 0 else 0)
dfconValF5['Neutral'] = dfconValF5['sentimiento'].apply(lambda x: 1 if x == 1 else 0)
dfconValF5['Positivo'] = dfconValF5['sentimiento'].apply(lambda x: 1 if x == 2 else 0)
#MOstrar solo las variables relevantes
dfconValF5 = dfconValF5[['yearsReviews','Negativo','Neutral','Positivo']]
#HAcer una agregacion por año, para Cada tipo, sea negativo,positivo y neutral 
agrupacionF5 = dfconValF5.groupby('yearsReviews').agg({
    'Negativo': 'sum',
    'Neutral': 'sum',
    'Positivo': 'sum'
}).reset_index()


In [25]:
agrupacionF5

,yearsReviews,Negativo,Neutral,Positivo
0,2010,10,8,48
1,2011,62,70,400
2,2012,2134,2285,6915
3,2013,784,1309,4699
4,2014,3394,4390,14327
5,2015,3448,3754,11268


In [60]:
#Exportacion de tabla de consulta f4
agrupacionF5.to_csv('agrupacionF5.csv', index=False)

In [61]:
def funcion5(valorF5):
    # Condicional si exite el numero de año
    if valorF5 in agrupacionF5['yearsReviews'].values :
    #Filtro de año
        # Igualar un filtro de año, solo al año a tomar
        añoFiltroF5 = agrupacionF5[agrupacionF5['yearsReviews'] == valorF5]
      
        # Crear lista de diccionarios con indices respectivos
        diccionariof5 = añoFiltroF5.drop(columns=['yearsReviews']).to_dict(orient='records')
        
        # Texto de salida funcio
        textoF5 = diccionariof5
    #Caso contrario 
    else:
        textoF5 = "Ingrese fecha Correcta"
    return textoF5

In [62]:
#Probar funcion 
funcion5(2011)

[{'Negativo': 62, 'Neutral': 70, 'Positivo': 400}]

In [12]:
agrupacionF1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   years             343 non-null    int64 
 1   genres            343 non-null    object
 2   playtime_forever  343 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 8.2+ KB


In [16]:
agrupacionF2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385874 entries, 0 to 385873
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           385874 non-null  object
 1   yearsReviews      385874 non-null  int32 
 2   genres            385874 non-null  object
 3   playtime_forever  385874 non-null  int64 
dtypes: int32(1), int64(1), object(2)
memory usage: 10.3+ MB


In [20]:
agrupacionF3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5561 entries, 0 to 5560
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   yearsReviews  5561 non-null   int32 
 1   sentimiento   5561 non-null   int64 
 2   item_name     5561 non-null   object
dtypes: int32(1), int64(1), object(1)
memory usage: 108.7+ KB


In [23]:
agrupacionF4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2205 entries, 0 to 2204
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   yearsReviews  2205 non-null   int32 
 1   sentimiento   2205 non-null   int64 
 2   item_name     2205 non-null   object
dtypes: int32(1), int64(1), object(1)
memory usage: 43.2+ KB


In [26]:
agrupacionF5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   yearsReviews  6 non-null      int32
 1   Negativo      6 non-null      int64
 2   Neutral       6 non-null      int64
 3   Positivo      6 non-null      int64
dtypes: int32(1), int64(3)
memory usage: 300.0 bytes
